# image re-labelling code
going over the images i have already labelled, and re-labelling them.

In [39]:
import numpy as np
import pandas as pd
import os
#from PIL import Image
from glob import glob
import csv
import matplotlib.pyplot as plt
from IPython.display import clear_output
import math
import shutil

from numba import jit
import h5py

In [4]:
## directory of set of images i would like to label + corresponding csv to populate
folder_path = '/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/'
# open csv quickly to check it is correct + in the format we expect
csv_file_name = '2ds_10000_8_dif.csv'
saving_csv_name = '2ds_10000_8_dif.csv'
csv_full_path = os.path.join(folder_path,csv_file_name)
full_save_csv_path = os.path.join(folder_path,saving_csv_name)
csv_read_pd = pd.read_csv(csv_full_path)
print(f'reading {csv_full_path}')
print(f'saving to {full_save_csv_path}')
n_images_done = csv_read_pd['2nd_label'].count()
print(f'number of images already labelled: {n_images_done}')
csv_read_pd.head()

reading /gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/2ds_10000_8_dif.csv
saving to /gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/2ds_10000_8_dif.csv
number of images already labelled: 8619


,image_name,number_label,2nd_label
0,1697685_23ch1,1.0,1.0
1,669294_20ch0,1.0,1.0
2,601343_19ch0,1.0,1.0
3,18058_26ch1,1.0,1.0
4,1261837_25ch0,3.0,NaN


In [3]:
## updating the already labelled csv - make new column + saving in og place
#csv_read_pd['2nd_label'] = np.nan
#csv_read_pd
#full_save_csv_path = os.path.join(folder_path,saving_csv_name)
#csv_read_pd.to_csv(full_save_csv_path, index=False)

## how many of each category
'''
count_cp = csv_read_pd['number_label'].value_counts().get(11.0, 0)
print(count_cp)
'''

"\ncount_cp = csv_read_pd['number_label'].value_counts().get(11.0, 0)\nprint(count_cp)\n"

# label things

In [13]:
number_of_interest = 11 # between 1 to 9, or 11

In [14]:
# okay we are going to need a completly different approach for this
def v2_saving_function(df,save_path):
    df.to_csv(save_path, index=False)
    print(f'{saving_csv_name} saved!')
    return df

# make a list of index where these images are located
interesting_idx = csv_read_pd.index[csv_read_pd['number_label'] == number_of_interest].tolist()

for i in range(len(interesting_idx)):
#for i in range(10):
    idx = interesting_idx[i]
    row = csv_read_pd.loc[idx]
    image_name = row.iloc[0]
    og_label = row.iloc[1]
    new_label = row.iloc[2]

    # I've already labelled it a second time
    if not pd.isna(new_label):
        continue

    ## lets do the whole image labelling thing
    else:
        print(image_name)
        im = Image.open(os.path.join(folder_path, image_name + '.png'))
        plt.figure(figsize=(8, 8))
        plt.imshow(im)
        plt.axis('off')
        plt.show()
        while True: # loop will continue until a valid entry is passed
            inp = input("Enter a label (0-9 or 11, or 'stop' to quit): ") ## label of image

            ## if write 'stop' will stop labelling + save what has been done
            if inp.lower() == 'stop':
                print("Stopped labeling.")
                updated_df = v2_saving_function(csv_read_pd,full_save_csv_path)
                raise SystemExit  # Exit gracefully
            
            try:
                num = int(inp) # convert to number (corresponding to label)
                if 0 <= num <= 11:  # Check if the number is in the valid range
                    clear_output(wait=True) # get rid of image once you've labelled it
                    csv_read_pd.loc[idx, '2nd_label'] = num ## as working in a df, we over-write the correct cell
                    break  # Exit the loop if the input is valid
                else:
                    print("Error: Label must be between 0 and 11.")
            except ValueError: # if we don't input a number -> stop labelling, so will break
                print("not words, what are you doing???")
        
updated_df = v2_saving_function(csv_read_pd,full_save_csv_path)

2ds_10000_8_dif.csv saved!


## hide extra stuff

In [7]:
csv_read_pd['2nd_label'].value_counts().get(1.0, 0)
#csv_read_pd['2nd_label'].count()
#df.to_csv(csv_file_list[0], index=False) 

2ds_10000_6.csv saved!


In [15]:
## copy newly labelled csv to github too
git_path = '/home/users/esree/CNN_basics/save_progress/'
shutil.copy(full_save_csv_path, git_path)

'/home/users/esree/CNN_basics/save_progress/2ds_10000_8_dif.csv'

# csv overwrite current label

In [5]:
og_csv = '2ds_10000.csv' ## original file
new_csv = '2ds_10000_8_dif.csv' ## new labels - in second column, but not all are second labelled
og_df = pd.read_csv(folder_path+og_csv)
new_df = pd.read_csv(folder_path+new_csv)

In [15]:
## combining og labels + new labels - replace old labels
length_of_data = len(og_df['image_name'])
name_lst = []
label_lst = []

for idx in range(int(length_of_data)):
    row = csv_read_pd.loc[idx]
    image_name = row.iloc[0]
    og_label = row.iloc[1]
    new_label = row.iloc[2]
    
    name_lst.append(image_name)
    if not pd.isna(new_label):
        label_lst.append(new_label)
    else:
        label_lst.append(og_label)

df_dic = {'image_name':name_lst,'number_label':label_lst}
new_df = pd.DataFrame(df_dic)
new_df

,image_name,number_label
0,1697685_23ch1,1.0
1,669294_20ch0,1.0
2,601343_19ch0,1.0
3,18058_26ch1,1.0
4,1261837_25ch0,3.0
...,...,...
9995,387342_02ch1,1.0
9996,676765_19ch0,1.0
9997,159619_06ch1,1.0
9998,2003638_23ch0,1.0


In [18]:
new_df['number_label'].count()
saving_csv_name = '2ds_10000_new.csv'
#saved_df = v2_saving_function(new_df,folder_path+saving_csv_name)

2ds_10000_new.csv saved!


# group images together to download them

In [44]:
folder_path = '/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/'
csv_file_name = '2ds_10000_new.csv'
csv_read_pd = pd.read_csv(folder_path+csv_file_name)

category_dic = {'1.0':'CP','2.0':'FA','3.0':'Co','4.0':'HPC','6.0':'CBC','7.0':'CA','8.0':'CC','9.0':'WD','0.0':'DIF','11.0':'unknown'}
category_dic_extra = {} # adding in path to loc 
for key in category_dic:
    var_name = category_dic[key]
    individ_cat_path = folder_path+var_name+'/'
    category_dic_extra[key] = [var_name,individ_cat_path]

'''
## make bunch of directories with appropriate names
category_names_lst = [category_dic[key] for key in category_dic] ## ayy make a list of category names from our dictionary values # newlist = [expression for item in iterable if condition == True]

# make each name a folder
for cat_name in category_names_lst:
    path = folder_path+cat_name+'/'
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"{cat_name} created successfully!")
    else:
        print(f"{cat_name} already exists.")
'''


length_of_data = len(csv_read_pd['image_name'])
for idx in range(int(length_of_data/1000)):
    row = csv_read_pd.loc[idx]
    image_name = row.iloc[0]
    label = row.iloc[1]
    og_loc = folder_path+image_name+'.png'
    new_loc = category_dic_extra[str(label)][1]
    
    shutil.copy(og_loc, new_loc)

print('copying images finished')

# zip up the folders with images in
for key in category_dic_extra:
    cat_folder = category_dic_extra[key][0]
    folder_path_to_zip = category_dic_extra[key][1][:-1]
    shutil.make_archive(folder_path_to_zip, 'zip',folder_path_to_zip) # as we are saving with same name but with .zip
    print(f'{cat_folder} zipped!')

CP
/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/CP
FA
/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/FA
Co
/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/Co
HPC
/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/HPC
CBC
/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/CBC
CA
/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/CA
CC
/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/CC
WD
/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/WD
DIF
/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/DIF
unknown
/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/unknown


In [29]:
category_names

['CP', 'FA', 'Co', 'HPC', 'CBC', 'CA', 'CC', 'WD', 'DIF', 'unknown']

In [40]:
csv_read_pd[:10]

,image_name,number_label
0,1697685_23ch1,1.0
1,669294_20ch0,1.0
2,601343_19ch0,1.0
3,18058_26ch1,1.0
4,1261837_25ch0,3.0
5,484599_25ch0,2.0
6,1435287_23ch0,1.0
7,1710083_20ch0,1.0
8,2586247_25ch1,1.0
9,299315_20ch0,1.0


# extract relevant particle stats of ones labelled

In [3]:
path_2ds_stats = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/2ds/all_2ds.csv'
path_labelled = '/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/2ds_10000_new_2nd.csv'

full_ds_stats = pd.read_csv(path_2ds_stats)
image_labelled_df = pd.read_csv(path_labelled)

images_labelled_lst = list(image_labelled_df['image_name'])

labelled_image_stats = full_ds_stats[full_ds_stats['name'].isin(images_labelled_lst)]


## need to add new column - with the label on
# not sure why 2 extra rows on df more than have been labelled??

## i have now discoved i have duplicate data, but i will have only extracted one of the images. but i don't know which one

In [9]:
duplicates_mask = labelled_image_stats.duplicated(subset=['name'])
duplicates_df = labelled_image_stats[duplicates_mask]
duplicates_df

,name,date,slice_s_idx,slice_e_idx,start_time,end_time,ellipse_d_max,Euclidean_d_max,Feret_d_max,area,...,first_diode_trunc,last_diode_trunc,image_trunc,aspect_ratio,wind_w,pres,alt,temp,LWC,IWC
855825,2484896_25ch1,220725,2484896,2484914,19:37:49.125000000,19:37:49.125000000,217.905812,184.390889,196.468827,5000.0,...,0,0,0,3.949269,-0.382931,380.90970,7979.3060,256.33566,0.006337,0.284747
940222,47102_07ch1,220807,47102,47131,19:26:38.926000000,19:26:39.519000000,289.030963,318.904374,328.024389,50300.0,...,0,0,0,1.172358,-4.348890,410.21915,7428.8936,257.82507,0.204330,0.008787


In [12]:
duplicate_1 = labelled_image_stats[labelled_image_stats['name'] == '2484896_25ch1']
duplicate_2 = labelled_image_stats[labelled_image_stats['name'] == '47102_07ch1']

duplicate_2

,name,date,slice_s_idx,slice_e_idx,start_time,end_time,ellipse_d_max,Euclidean_d_max,Feret_d_max,area,...,first_diode_trunc,last_diode_trunc,image_trunc,aspect_ratio,wind_w,pres,alt,temp,LWC,IWC
923312,47102_07ch1,220807,47102,47126,16:37:00.357000000,16:37:03.899000000,273.403521,282.842712,290.688837,41600.0,...,18,0,0,1.283139,0.561548,446.08530,6787.3706,261.83868,0.001382,0.000019
940222,47102_07ch1,220807,47102,47131,19:26:38.926000000,19:26:39.519000000,289.030963,318.904374,328.024389,50300.0,...,0,0,0,1.172358,-4.348890,410.21915,7428.8936,257.82507,0.204330,0.008787


In [2]:
new_csv = '/gws/nopw/j04/dcmex/users/ezriab/image_labelling/2ds_10000_sample/2ds_10000_new_2nd.csv'
full_ds_stats = pd.read_csv(new_csv)
 

full_ds_stats['note'].count()

SyntaxError: invalid syntax (3767629083.py, line 3)

In [16]:


full_duplicates_mask = full_ds_stats.duplicated(subset=['name'])
full_duplicates_df = full_ds_stats[full_duplicates_mask]
full_duplicates_df

list_of_duplicates = list(full_duplicates_df['name'])
double_duplicates_df = full_ds_stats[full_ds_stats['name'].isin(list_of_duplicates)]

double_duplicates_df

,name,date,slice_s_idx,slice_e_idx,start_time,end_time,ellipse_d_max,Euclidean_d_max,Feret_d_max,area,...,first_diode_trunc,last_diode_trunc,image_trunc,aspect_ratio,wind_w,pres,alt,temp,LWC,IWC
3327,94298_19ch0,220719,94298,94332,18:53:04.977000000,18:53:04.977000000,349.710133,392.045916,402.243707,71300.0,...,25,0,0,1.219161,-2.043195,400.87717,7644.9404,257.90890,0.032683,0.076399
3328,94298_19ch0,220719,94298,94311,18:53:04.977000000,18:53:04.977000000,146.839578,130.000000,140.356688,4400.0,...,0,0,0,3.089049,-2.043195,400.87717,7644.9404,257.90890,0.032683,0.076399
8430,226470_19ch0,220719,226470,226482,18:56:50.694000000,18:56:50.694000000,152.272582,126.491106,143.178211,4200.0,...,0,0,0,3.395385,-2.762317,388.69946,7885.3310,255.70236,0.162223,0.334879
8431,226470_19ch0,220719,226470,226483,18:56:50.694000000,18:56:50.694000000,174.590712,172.046505,186.010752,13000.0,...,0,0,0,1.533949,-2.762317,388.69946,7885.3310,255.70236,0.162223,0.334879
8595,230869_19ch0,220719,230869,230879,18:56:50.772000000,18:56:50.772000000,135.556367,144.222051,152.643375,8000.0,...,0,0,0,1.263412,-2.762317,388.69946,7885.3310,255.70236,0.186962,0.442953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
932424,315316_07ch1,220807,315316,315356,17:34:41.285000000,17:34:41.300000000,417.369190,512.249939,520.096145,88900.0,...,0,33,0,1.315120,-1.812491,416.33398,7310.0000,258.07930,0.019274,0.000000
939967,18539_07ch1,220807,18539,18595,19:18:02.117000000,19:18:02.117000000,579.097229,676.239603,688.186021,146400.0,...,6,0,0,1.589710,0.090431,431.38864,7046.8150,260.83000,0.000063,0.000000
940010,34578_07ch1,220807,34578,34691,19:20:07.116000000,19:20:40.110000000,977.017933,1136.353818,1146.298390,223500.0,...,0,83,0,3.120222,-3.798381,417.02188,7294.1480,259.73830,0.000284,0.000012
940011,34578_07ch1,220807,34578,34603,19:20:07.116000000,19:20:40.110000000,258.089121,259.615100,269.258240,8200.0,...,0,17,0,4.987685,-3.798381,417.02188,7294.1480,259.73830,0.000284,0.000012


In [8]:
length_of_data = len(full_ds_stats['image_name'])

for idx in range(length_of_data):
    row = full_ds_stats.loc[idx]
    image_name = row.iloc[0]
    label = row.iloc[1]
    note = row.iloc[2]
    #og_loc = folder_path+image_name+'.png'
    #new_loc = category_dic_extra[str(label)][1]
    if not note != 'a':
        print('yep')
	    #shutil.copy(og_loc, new_loc)

yep
yep
yep
yep
yep
yep
yep
yep
yep
yep
yep


In [29]:
full_duplicates_df

#double_duplicates_df.to_csv('duplicates_df.csv', index=False)
reduced_duplicates = full_duplicates_df[:10]
reduced_duplicates.to_csv('/gws/nopw/j04/dcmex/users/ezriab/processed_stats/silly_duplicates/red_duplicates_df.csv', index=False)

## fixing that because i am stupid

In [68]:
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from glob import glob
import seaborn as sns
import pandas as pd
from scipy.special import gamma
from datetime import datetime, timedelta

import h5py ####
from PIL import Image
import os
import psutil

from scipy.ndimage import convolve, label
from skimage.measure import regionprops, find_contours
from scipy.spatial import ConvexHull, distance_matrix
from scipy.spatial.distance import pdist
from skimage.morphology import remove_small_holes ## remove holes <3
from scipy.ndimage import binary_fill_holes
from skimage import measure
import tensorflow as tf

from numba import jit

In [69]:
## og functions / things to make script run smoothly
## setting thresholds / res for attaining good particle final images
fill_hole_threshold = 5 # max number pixels contained within particle that is filled in

minimum_area = 20 # very quick metric to stop the processing of particles with area < 15 pixels
desired_image_size = 200 
pixel_resolution = 10 # mu for 2DS // need to be 150 for hvps
length_threshold = 100 ## 100 mu for 2ds // 1500 mu for hvps

def stats_description(bw_crystal, fill_hole_thresh):
    #take binary image, fill in small holes and returns object containing stats about crystal
    
    filled_particle = remove_small_holes(bw_crystal.image, area_threshold=fill_hole_thresh) # fill in voids within binary image - better estimation of stats # may need to be altered
       
    if filled_particle.shape[0] < 2 or filled_particle.shape[1] < 2:
        return filled_particle, None
        
    contours = measure.find_contours(filled_particle, 0.5)
    if contours:
        contour = max(contours, key=lambda x: x.shape[0])  # Sort contours by area (largest first) and select the largest contour
        
        labeled_image = measure.label(filled_particle)  # Label the image based on the threshold
        region = measure.regionprops(labeled_image)[0]  # Assumes largest labeled region corresponds to largest contour
        
        return filled_particle, region
    else:
        return filled_particle, None


## function to calculate truncation of particle
@jit(nopython=True) # Enables full optimization by numba
def calc_truncation(particle_coords):
    ## so much simpler, looking at list of coordinates making up a particle, then summing ones in 0 and 127 row - i.e. first + last diode
    lst_first_diode = [coord for coord in particle_coords if coord[0] == 0]
    lst_last_diode = [coord for coord in particle_coords if coord[0] == 127]

    n_top = len(lst_first_diode)
    n_bottom = len(lst_last_diode)

    return n_top, n_bottom # number pixels touching top / bottom respectively
## 

###  set up dataframe, used to extract from raw h5 file + has stats about the particle
columns = [
    "name",
    "date",
    "slice_s_idx",
    "slice_e_idx",
    #"start_time", #hh:mm:ss 
    #"end_time", #hh:mm:ss 
    "ellipse_d_max", # um
    #"ellipse_d_min", # um
    "Euclidean_d_max", # um
    "Feret_d_max", # um
    "area", # um2
    "perimeter", # um
    "circularity",
    "probe",
    "first_diode_trunc",
    "last_diode_trunc",
    "image_trunc",
    "aspect_ratio"
    ]


In [76]:
#dup_csv_pth = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/silly_duplicates/red_duplicates_df.csv'
dup_csv_pth = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/silly_duplicates/duplicates_df.csv'
save_loc = '/gws/nopw/j04/dcmex/users/ezriab/processed_stats/silly_duplicates/'

base_h5_pth = '/gws/nopw/j04/dcmex/users/ezriab/raw_h5/2ds/'

particle_df = pd.DataFrame(columns=columns)
with open(dup_csv_pth, mode='r') as file:
    csv_reader = csv.reader(file)
    header = next(csv_reader)  # Read the header row
    # loop through each row
    for row in csv_reader:
        image_name = row[0]
        date = row[1] 
        start_idx = row[2]
        end_idx = row[3]
        probe = row[12]
        date_day = date[-2:]
        
        h5_of_interest = [] ## some have multiple files associated with each date
        list_h5_files = glob(base_h5_pth+probe+'/'+'*.h5')
        for file in list_h5_files:
            if date in file:
                h5_of_interest.append(file)

        for h5_path in h5_of_interest:
            h5_file = h5py.File(h5_path,'r')

            try:
                h5_image = h5_file['ImageData']
                h5_time = h5_file['ImageTimes']
            except KeyError as e:
                print(f"Dataset missing in file: {file}. Error: {e}")
                continue

            one_crystal = h5_file['ImageData'][:,int(start_idx)-2:int(end_idx)+2] # extract 1 crystal
            #plt.imshow(one_crystal, cmap='gray')
            binary_image = (one_crystal == 0) 
            labeled_image, num_features = label(binary_image)
            try:
                props = regionprops(labeled_image)
            except ValueError as e:
                print('something has gone wrong')
                continue
                
            for particle in props:
                if particle.area >= minimum_area:
                    filled_part, spec_region = stats_description(particle,fill_hole_threshold)
                    if spec_region:
                        aspect_ratio_value = spec_region.major_axis_length / spec_region.minor_axis_length
                        coords = particle.coords # basically gives coords of each point of interest [row,column]
                        distances = pdist(coords)
                        euclidean_dim = np.max(distances)

                        if euclidean_dim * pixel_resolution >= length_threshold:
                                
                            ## basic info
                            x_values = np.unique(coords[:, 1])
                            #s_idx = int(selected_pix_sum[i] + x_values[0])
                            #e_idx = int(selected_pix_sum[i] + x_values[-1])
            
                            ## truncation calc
                            # cnn - in final pic size, particle may be truncated if very long, this tells us how many pixels may be cut off 
                            image_trunc = x_values[-1] - desired_image_size 
                            if image_trunc < 0:
                                image_trunc = 0
                            # normal trunc calculation - on actual probe
                            first_diode, last_diode = calc_truncation(coords)
                            
                            ## using circularity calculation from Crosier et al. 2011
                            circularity_calc = np.divide((spec_region.perimeter**2),(4*np.pi*spec_region.area))
                            particle_name = f'{start_idx}_{particle.label}_{date_day}{probe}'
                            
                            # nice way of saving data - lenth + measurements are correct in microns
                            one_particle_data = {
                                    #"image_index": image_index,
                                    "name": particle_name,
                                    "date" : date,
                                    "slice_s_idx": start_idx,
                                    "slice_e_idx": end_idx,
                                    #"start_time": str(selected_utc_time[i].values).split('T')[1], # more friendly time
                                    #"end_time": str(selected_utc_time[i+1].values).split('T')[1], # more friendly time
                                    "ellipse_d_max": spec_region.major_axis_length * pixel_resolution, ## d_max (equivalent ellipse)
                                    #"ellipse_d_min": spec_region.minor_axis_length * pixel_resolution, ## d_min (equivalent ellipse)
                                    "Euclidean_d_max": euclidean_dim * pixel_resolution,
                                    "Feret_d_max":spec_region.feret_diameter_max * pixel_resolution,
                                    "area": (spec_region.area * (pixel_resolution**2)),
                                    "perimeter": (spec_region.perimeter * pixel_resolution),
                                    "circularity": circularity_calc,
                                    "probe": probe,
                                    "first_diode_trunc": first_diode,
                                    "last_diode_trunc": last_diode,
                                    "image_trunc": image_trunc,
                                    "aspect_ratio": aspect_ratio_value  
                                    }

                            filled_part = filled_part.astype(np.float32) ## convert to float 0 and 1s
                            filled_part = np.expand_dims(filled_part, axis=-1) ## add extra dimention - this is for adding padding
            
                            imagex = tf.image.resize_with_crop_or_pad(filled_part, desired_image_size, desired_image_size)
                            ## save image
                            # Remove the extra dimension if needed
                            image_np = imagex.numpy().squeeze()
                            #plt.imshow(image_np, cmap='gray')
                            plt.imsave(f'{save_loc}{particle_name}.png', image_np, cmap="gray")
                            

            h5_file.close()
            #print(one_particle_data)
            one_particle_data_df = pd.DataFrame([one_particle_data])
            particle_df = pd.concat([particle_df, one_particle_data_df], ignore_index=True)

particle_df.to_csv(f'{save_loc}updated_duplicates.csv', index=False)        
        

/tmp/ipykernel_582/2547753393.py:111: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  particle_df = pd.concat([particle_df, one_particle_data_df], ignore_index=True)


Dataset missing in file: /gws/nopw/j04/dcmex/users/ezriab/raw_h5/2ds/ch0/Export_base220808160342.h5. Error: "Unable to synchronously open object (object 'ImageData' doesn't exist)"
Dataset missing in file: /gws/nopw/j04/dcmex/users/ezriab/raw_h5/2ds/ch0/Export_base220808160342.h5. Error: "Unable to synchronously open object (object 'ImageData' doesn't exist)"
Dataset missing in file: /gws/nopw/j04/dcmex/users/ezriab/raw_h5/2ds/ch0/Export_base220808160342.h5. Error: "Unable to synchronously open object (object 'ImageData' doesn't exist)"
Dataset missing in file: /gws/nopw/j04/dcmex/users/ezriab/raw_h5/2ds/ch0/Export_base220808160342.h5. Error: "Unable to synchronously open object (object 'ImageData' doesn't exist)"
something has gone wrong
something has gone wrong
something has gone wrong
something has gone wrong
something has gone wrong
something has gone wrong
something has gone wrong
something has gone wrong
something has gone wrong
something has gone wrong
something has gone wrong
s

In [77]:
particle_df

,name,date,slice_s_idx,slice_e_idx,ellipse_d_max,Euclidean_d_max,Feret_d_max,area,perimeter,circularity,probe,first_diode_trunc,last_diode_trunc,image_trunc,aspect_ratio
0,94298_2_19ch0,220719,94298,94332,146.839578,130.000000,140.356688,4400.0,313.137085,1.773399,ch0,0,0,0,3.089049
1,94298_2_19ch0,220719,94298,94311,146.839578,130.000000,140.356688,4400.0,313.137085,1.773399,ch0,0,0,0,3.089049
2,226470_3_19ch0,220719,226470,226482,174.590712,172.046505,186.010752,13000.0,652.487373,2.606100,ch0,0,0,0,1.533949
3,226470_5_19ch0,220719,226470,226483,174.590712,172.046505,186.010752,13000.0,652.487373,2.606100,ch0,0,0,0,1.533949
4,230869_5_19ch0,220719,230869,230879,127.331888,116.619038,125.299641,6300.0,403.847763,2.060084,ch0,0,0,0,1.422839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,34578_7_07ch1,220807,34578,34691,258.089121,259.615100,269.258240,8200.0,519.350288,2.617562,ch1,0,17,0,4.987685
305,34578_3_07ch1,220807,34578,34603,296.417202,334.215499,342.052628,58200.0,1009.055916,1.392187,ch1,0,27,0,1.131847
306,34578_7_07ch1,220807,34578,34603,258.089121,259.615100,269.258240,8200.0,519.350288,2.617562,ch1,0,17,0,4.987685
307,47102_1_07ch1,220807,47102,47131,317.255314,344.093011,355.105618,48600.0,1407.401154,3.243319,ch1,20,0,0,1.307888
